# Histogramming, grouping, and binning

## Overview

Histogramming (see [sc.histogram](../../generated/scipp.histogram.rst#scipp.histogram)), grouping (using [sc.groupby](../groupby.ipynb)), and binning (see [Binned data](binned-data.ipynb)) all serve similar but slightly different purposes.
Picking the optimal one of the three for a particular application may yield more natural code and better performance.
Let us start by an example.
Consider a table of scattered measurements:

In [ ]:
import numpy as np
import scipp as sc
from scipp.plot import plot
N = 5000
values = 10*np.random.rand(N)
data = sc.DataArray(
    data=sc.Variable(dims=['position'], unit=sc.units.counts, values=values, variances=values),
    coords={
        'x':sc.Variable(dims=['position'], unit=sc.units.m, values=np.random.rand(N)),
        'y':sc.Variable(dims=['position'], unit=sc.units.m, values=np.random.rand(N))
    })
data.values *= 1.0/np.exp(5.0*data.coords['x'].values)
sc.table(data['position', :5])

We may now be interested in the total intensity (counts) as a function of `'x'`.
There are three ways to do this:

In [ ]:
xbins = sc.Variable(dims=['x'], unit=sc.units.m, values=np.linspace(0,1,num=40))
ds = sc.Dataset()
ds['histogram'] = sc.histogram(data, xbins)
ds['groupby']   = sc.groupby(data, group='x', bins=xbins).sum('position')
ds['bin']       = sc.bin(data, edges=[xbins]).bins.sum()
plot(ds)

In the above plot we can only see a single line, since the three solutions yield exactly the same result (neglecting floating-point rounding errors):

- `histogram` sorts data points into 'x' bins, summing immediately.
- `groupby` groups by 'x' and then sums (on-the-fly) all data points falling in the same 'x' bin.
- `bin` sorts data points into 'x' bins.
  Summing all rows in a bin yields the same result as grouping and summing directly.

So in this case we get equivalent results, but the application areas differ, as described in more detail in the following sections.

## Histogramming

`histogram` directly sums the data and is efficient.
Limitations are:

- Can currently only histogram along a single dimension.
- Can currently only apply the "sum" to accumulate into a bin.
  Support for more operatinos, such as "mean", is considered for the future.
  
While histogramming is only supported along a single dimension, we *can* histogram binned data (since [binning](#Binning) preserves the `'y'` coord), to create 2-D (or N-D) histograms:

In [ ]:
binned = sc.bin(data, edges=[xbins])
ybins = sc.Variable(dims=['y'], unit=sc.units.m, values=np.linspace(0,1,num=30))
hist = sc.histogram(binned, ybins)
plot(hist)

In [ ]:
hist

Another capability of `histogram` is to histogram a dimension that has previously been binned with a different or higher resolution, i.e. different bin edges.
Compare to the plot of the initial example:

In [ ]:
binned = sc.bin(data, edges=[xbins])
xbins_fine = sc.Variable(dims=['x'], unit=sc.units.m, values=np.linspace(0,1,num=100))
plot(sc.histogram(binned, xbins_fine))

## Grouping

`groupby` is more flexible in terms of operations than can be applied and may be the go-to solution when a quick one-liner is required.
Limitations are:

- Can only group along a single dimension.
- Works best for small to medium-sized data, or if data is already mostly sorted along the grouping dimension.
  Slow if millions of small input slices contribute to each group.
  
`groupby` can also operate on binned data, combining bin contents by concatenation:

In [ ]:
binned = sc.bin(data, edges=[xbins])
binned.coords['param'] = sc.Variable(dims=['x'], values=(np.random.random(39)*4).astype(np.int32))
grouped = sc.groupby(binned, 'param').bins.concatenate('x')
grouped

Each output bin is a combination of multiple input bins:

In [ ]:
grouped.values[0]

## Binning
  
`bin` actually reorders data and meta data such that all data contributing to a bin is in a contiguous block.
Binning along multiple dimensions is supported.
Of the three options it is the only solution that supports *modifying* data in the grouped/binned layout.
A variety of operations on such binned data is available.
Limitations are:

- Requires copying and reordering the input data and can thus become expensive.
  
In the above example the `'y'` information is dropped by `histogram` and `groupby`, but `bin` preserves it:

In [ ]:
binned = sc.bin(data, edges=[xbins])
binned.values[0]

If we omit the call to `bins.sum` in the original example, we can subsequently apply another [histogramming](#Histogramming) or binning operation to the data:

In [ ]:
binned = sc.bin(binned, edges=[ybins])
binned

As in the 1-D example above, summing the bins is equivalent to histogramming binned data:

In [ ]:
plot(binned.bins.sum())